In [1]:
from icecube import icetray, dataio, dataclasses, simclasses, clsim
from icecube.icetray import I3Units, OMKey, I3Frame
from icecube.dataclasses import ModuleKey
from os.path import expandvars
import numpy as np
import matplotlib.pylab as plt
import argparse
from scipy import stats
from scipy.optimize import minimize
from scipy.stats.distributions import chi2
import scipy
from RecoPulseLikelihoodRatio import likelihoodfit
from likelihoodHelpers import log_likelihood_biGauss, log_likelihood_doublePeak, likelihood_ratio_doublePeak, likelihood_ratio_biGauss, biGauss, double_peak
import csv

In [5]:
read_file = open('/data/p-one/akatil/analysis/RecoPulses/RecoPulseFitInfo_correcectedchi2.csv', 'r')

file_num, frame_num, lepton, DOM, string = ([]), ([]), ([]), ([]), ([])
binEntries_mean, success_biGauss, success_doublePeak = ([]), ([]), ([])
biGauss_pos, biGauss_wid, biGauss_rat, biGauss_amp = ([]), ([]), ([]), ([])
doublePeak_amp1, doublePeak_amp2, doublePeak_pos1, doublePeak_pos2 = ([]), ([]), ([]), ([])
doublePeak_rat1, doublePeak_rat2, doublePeak_wid1, doublePeak_wid2 = ([]), ([]), ([]), ([])
area_data, area_biGauss_fit, area_doublePeak_fit = ([]), ([]), ([])
gof_biGauss, gof_doublePeak = ([]), ([])
with read_file:

    reader = csv.DictReader(read_file)
    
    for row in reader:
        file_num = np.append(file_num, row['file'])
        frame_num = np.append(frame_num, row['frame'])
        lepton =np.append(lepton, row['lepton_type'])
        DOM = np.append(DOM, row['DOM']) #DOM and string have been interchanged
        string = np.append(string, row['string'])
        binEntries_mean = np.append(binEntries_mean, float(row['binEntries_mean']))
        success_biGauss = np.append(success_biGauss, row['success_biGauss'])
        success_doublePeak = np.append(success_doublePeak, row['success_doublePeak'])
        biGauss_pos = np.append(biGauss_pos, float(row['biGauss_pos']))
        biGauss_wid = np.append(biGauss_wid, float(row['biGauss_wid']))
        biGauss_rat = np.append(biGauss_rat, float(row['biGauss_rat']))
        biGauss_amp = np.append(biGauss_amp, float(row['biGauss_amp']))
        doublePeak_pos1 = np.append(doublePeak_pos1, float(row['doublePeak_pos1']))
        doublePeak_wid1 = np.append(doublePeak_wid1, float(row['doublePeak_wid1']))
        doublePeak_rat1 = np.append(doublePeak_rat1, float(row['doublePeak_rat1']))
        doublePeak_amp1 = np.append(doublePeak_amp1, float(row['doublePeak_amp1']))
        doublePeak_pos2 = np.append(doublePeak_pos2, float(row['doublePeak_pos2']))
        doublePeak_wid2 = np.append(doublePeak_wid2, float(row['doublePeak_wid2']))
        doublePeak_rat2 = np.append(doublePeak_rat2, float(row['doublePeak_rat2']))
        doublePeak_amp2 = np.append(doublePeak_amp2, float(row['doublePeak_amp2']))
        area_data = np.append(area_data, float(row['area_data']))
        area_biGauss_fit = np.append(area_biGauss_fit, float(row['area_biGauss_fit']))
        area_doublePeak_fit = np.append(area_doublePeak_fit, float(row['area_doublePeak_fit']))
        gof_biGauss = np.append(gof_biGauss, float(row['gof_biGauss']))
        gof_doublePeak = np.append(gof_doublePeak, float(row['gof_doublePeak']))

In [6]:
'''
Selecting DOMs with goodness of fit < 2, only double peak fit chi2 values are considered
'''
gof_bool_biGauss = (gof_biGauss > 0)&(gof_biGauss < 2)
gof_bool_dp = (gof_doublePeak > 0)&(gof_doublePeak < 2)

select_gof_biGauss = gof_biGauss[gof_bool_dp]
select_gof_dp = gof_doublePeak[gof_bool_dp]
select_file_num = file_num[gof_bool_dp]
select_frame_num = frame_num[gof_bool_dp]
select_string = DOM[gof_bool_dp]
select_DOM = string[gof_bool_dp]

/cvmfs/icecube.opensciencegrid.org/py2-v3.1.1/RHEL_7_x86_64/lib/python2.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in greater
  """
/cvmfs/icecube.opensciencegrid.org/py2-v3.1.1/RHEL_7_x86_64/lib/python2.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in less
  """


In [7]:
gof_biGauss

array([12.9562574 ,  1.94526641,  9.48373854, ..., 14.77778603,
       10.41280474,  8.01624411])

In [ ]:
'''
plotting the histograms
'''

for i in range(50, 70):
#for i in select_file_num[10:40]:
    print(select_file_num[i])
    file_name = dataio.I3File('/data/p-one/akatil/step_5_medium_water/NuTau_NuE_20Events/step_5_'+str(select_file_num[i])+'_medium_water_custom_mDOM_recoPulse.i3.gz')
    #print('/data/p-one/akatil/step_5_medium_water/NuTau_NuE_20Events/step_5_'+str(select_file_num[i])+'_medium_water_custom_mDOM_recoPulse.i3.gz')
    frameList = []
    j = 0
    while(file_name.more()):
        frameList.append(file_name.pop_daq())
    frame_number = int(select_frame_num[i])
    frame_i = frameList[frame_number]
    mctree = frame_i["I3MCTree"]
    primary = mctree.primaries
    lepton = dataclasses.I3MCTree.first_child(mctree, primary[0].id)    
    
    omkey = OMKey(int(select_string[i]), int(select_DOM[i]), 0)
    print(omkey)
    recoPulseMap = frame_i['I3RecoPulses']
    recoPulseList = recoPulseMap[omkey]
    recoPulse_timeList = np.array([recoPulse.time for recoPulse in recoPulseList])
    recoPulse_chargeList = np.array([recoPulse.charge for recoPulse in recoPulseList])
    
    '''
    Removing DOMs with hits less than 100 Hits
    '''
    if sum(recoPulse_chargeList) < 100:
        print(sum(recoPulse_chargeList))
        print('exit1')
        continue

    '''
    Calculating the mean and removing the tails
    '''

    #mean = recoPulse_timeList.mean()
    mean = sum(recoPulse_timeList*recoPulse_chargeList)/sum(recoPulse_chargeList) #mean is weighted
    select_time = recoPulse_timeList[(recoPulse_timeList > mean-50) & (recoPulse_timeList < mean+50)]
    select_charge = recoPulse_chargeList[(recoPulse_timeList > mean-50) & (recoPulse_timeList < mean+50)]
    #print('SELECT CHARGE', select_charge, select_time, mean, recoPulse_timeList, recoPulse_chargeList)

    if len(select_time) < 10:
        print('exit2')
        continue

    mean_select_time = sum(select_time*select_charge)/sum(select_charge)
    max_hitTimes = recoPulse_timeList[(recoPulse_timeList > (mean_select_time-100))&(recoPulse_timeList < (mean_select_time+100))]
    max_charge = recoPulse_chargeList[(recoPulse_timeList > (mean_select_time-100))&(recoPulse_timeList < (mean_select_time+100))]

    #[using zscore to remove the effect of outliers from the analysis]
    z = stats.zscore(max_hitTimes)
    max_hitTimes = max_hitTimes[(z>-1.6)&(z < 1.2)]
    max_charge = max_charge[(z>-1.6)&(z < 1.2)]

    if len(max_hitTimes) < 10:
        print('exit3')
        continue

    #Shifting mean to zero
    max_hitTimes_mean = sum(max_hitTimes*max_charge)/sum(max_charge)
    timestamps = max_hitTimes - max_hitTimes_mean
    final_mean = timestamps.mean()

    '''
    Histogramming the data from simulation
    '''
    print('Now Histogramming')
    bins = np.arange(min(timestamps), max(timestamps), 1)
    num, bin_edges = np.histogram(timestamps, bins=bins, weights=max_charge)
    bin_centers = (bin_edges[:-1]+bin_edges[1:])/2
                    
    #removing bins with < 0 entries ---> Recommended for binned likelihood
    entries_in_bins = num[num > 0]
    bin_centers = bin_centers[num > 0]

    #Degrees of freedom should be greater than zero!
    if len(entries_in_bins) < 8:
        continue

    #Getting data for the chi2 fit
    chi2_entries_in_bins = entries_in_bins[entries_in_bins > 10]
    chi2_bin_centers = bin_centers[entries_in_bins > 10]

    num_dataPoints = len(entries_in_bins)
    area_data = sum(entries_in_bins)
    mean_entries = entries_in_bins.mean()

    '''
    Fitting bifurcated Gaussian and double bifurcated gaussian to
    the mcpe hit time distributions for both tau and electron.
    '''

    #Single Peak

    nll = lambda *args: log_likelihood_biGauss(*args)
    initial_biGauss = np.array([final_mean, 50, 5, max(entries_in_bins)])
    bnds_biGauss = ((min(bin_centers), max(bin_centers)), (0, 500), (0, 10), (0, 1e6))
    soln_biGauss = minimize(log_likelihood_biGauss, initial_biGauss,
                                    args=(entries_in_bins, bin_centers),
                                    method='Powell',
                                    bounds = bnds_biGauss)

    #Double Peak

    nll = lambda *args: log_likelihood_doublePeak(*args)
    initial_doublePeak = np.array([min(bin_centers)+10, 20, 1, max(entries_in_bins), final_mean, 20, 1, max(entries_in_bins)])
    bnds_doublePeak = ((min(bin_centers), final_mean), (0, 500), (0, 10), (0, 1e6),
                        (final_mean, max(bin_centers)), (0, 500), (0, 10), (0,1e6))
    soln_doublePeak = minimize(log_likelihood_doublePeak, initial_doublePeak,
                                args=(entries_in_bins, bin_centers),
                                method='Powell',
                                bounds=bnds_doublePeak)
    
    LR_biGauss = likelihood_ratio_biGauss(bin_centers, entries_in_bins, soln_biGauss.x[0],
                                              soln_biGauss.x[1], soln_biGauss.x[2], soln_biGauss.x[3])
    LR_doublePeak = likelihood_ratio_doublePeak(bin_centers, entries_in_bins, soln_doublePeak.x[0],
                                                    soln_doublePeak.x[1],soln_doublePeak.x[2],
                                                    soln_doublePeak.x[3], soln_doublePeak.x[4],
                                                    soln_doublePeak.x[5], soln_doublePeak.x[6],
                                                    soln_doublePeak.x[7])
    
    '''
    goodness of fit - Chi2 = 2*ln(LR)
    '''
    gof_DOM_biGauss = (2*LR_biGauss)/(num_dataPoints - 4)
    gof_DOM_doublePeak = (2*LR_doublePeak)/(num_dataPoints - 8)
    
    if gof_DOM_doublePeak != gof_DOM_doublePeak:
        print('PARAMETERS -', soln_doublePeak.x[0], soln_doublePeak.x[1],soln_doublePeak.x[2],
                                                    soln_doublePeak.x[3], soln_doublePeak.x[4],
                                                    soln_doublePeak.x[5], soln_doublePeak.x[6],
                                                    soln_doublePeak.x[7])
        
        def log_likelihood_doublePeak_test(theta, n, x):
            pos1, wid1, r1, amp1, pos2, wid2, r2, amp2 = theta
            model = double_peak(x, pos1, wid1, r1, amp1, pos2, wid2, r2, amp2)
            L = model - (n*np.log(model))
            print('*****************Double Peak***************', int(i), omkey)
            print(theta, np.sum(L))
            return np.sum(L)
        
        
        soln_doublePeak_test = minimize(log_likelihood_doublePeak_test, initial_doublePeak,
                            args=(entries_in_bins, bin_centers),
                            method='Powell',
                            bounds=bnds_doublePeak)
    
        '''
        (x, y) values for the fit
        '''
        #x = bin_centers
        x = np.linspace(min(bin_centers), max(bin_centers), 1000)
        y_biGauss = biGauss(x, soln_biGauss.x[0],
                            soln_biGauss.x[1], soln_biGauss.x[2], soln_biGauss.x[3])
        y_doublePeak = double_peak(x, soln_doublePeak.x[0], soln_doublePeak.x[1],
                                        soln_doublePeak.x[2], soln_doublePeak.x[3], soln_doublePeak.x[4],
                                        soln_doublePeak.x[5], soln_doublePeak.x[6], soln_doublePeak.x[7])
    
    
                    
        plt.figure(figsize=(10,9))
        _ = plt.hist(timestamps, bins=bins, weights=max_charge, histtype='step')
        #plt.plot(x, y_biGauss, '-', c = 'k')
        plt.plot(x, y_doublePeak, '-', c = 'r')
        plt.title(str(lepton.type) +' Double Peak(Chi2/dof)- '+ str(select_gof_dp[i])+' LR - '+str(gof_DOM_doublePeak))
        
        break

1392
OMKey(9,19,0)
Now Histogramming
1420
OMKey(2,4,0)
Now Histogramming
1461
OMKey(4,4,0)
Now Histogramming
1465
OMKey(4,5,0)
Now Histogramming
1470
OMKey(8,18,0)
Now Histogramming
1548
OMKey(5,1,0)
Now Histogramming
1587
OMKey(3,10,0)
Now Histogramming
1626
OMKey(7,17,0)
Now Histogramming


In [ ]:
# frame_in_file = select_frame_num[select_file_num == i]
    frame_number = 0
    for frame in file:
        for j in frame_in_file:
            if frame_number == int(j):
                mctree = frame["I3MCTree"]
                primary = mctree.primaries
                lepton = dataclasses.I3MCTree.first_child(mctree, primary[0].id)
                
                recoPulseMap = frame['I3RecoPulses']
                select_string_in_frame = select_string[(select_file_num == i) & (select_frame_num == j)]
                
                for k in select_string_in_frame:
                    select_DOM_with_string = select_DOM[(select_file_num == i) & (select_frame_num == j) & (select_string == k)]
                    omkey = OMKey(int(k), int(select_DOM_with_string[0]), 0)
                    print('OMKEY', omkey)
                    print(num)
                    print(i, j, k, int(select_DOM_with_string[0]))
                    print(i, select_frame_num[num], select_string[num], select_DOM[num])
                    select_gof_DOM_biGauss = select_gof_biGauss[(select_file_num == i) & (select_frame_num == j) & (select_string == k)& (select_DOM == select_DOM_with_string[0])]
                    select_gof_DOM_dp = select_gof_dp[(select_file_num == i) & (select_frame_num == j) & (select_string == k)& (select_DOM == select_DOM_with_string[0])]
                    
                    recoPulseList = recoPulseMap[omkey]
                    recoPulse_timeList = np.array([recoPulse.time for recoPulse in recoPulseList])
                    recoPulse_chargeList = np.array([recoPulse.charge for recoPulse in recoPulseList])
                    
                    '''
                    Removing DOMs with hits less than 100 Hits
                    '''
                    if sum(recoPulse_chargeList) < 100:
                        print('exit1')
                        continue

                    '''
                    Calculating the mean and removing the tails
                    '''

                    #mean = recoPulse_timeList.mean()
                    mean = sum(recoPulse_timeList*recoPulse_chargeList)/sum(recoPulse_chargeList) #mean is weighted
                    select_time = recoPulse_timeList[(recoPulse_timeList > mean-50) & (recoPulse_timeList < mean+50)]
                    select_charge = recoPulse_chargeList[(recoPulse_timeList > mean-50) & (recoPulse_timeList < mean+50)]
                    #print('SELECT CHARGE', select_charge, select_time, mean, recoPulse_timeList, recoPulse_chargeList)

                    if len(select_time) < 10:
                        print('exit2')
                        continue

                    mean_select_time = sum(select_time*select_charge)/sum(select_charge)
                    max_hitTimes = recoPulse_timeList[(recoPulse_timeList > (mean_select_time-100))&(recoPulse_timeList < (mean_select_time+100))]
                    max_charge = recoPulse_chargeList[(recoPulse_timeList > (mean_select_time-100))&(recoPulse_timeList < (mean_select_time+100))]

                    #[using zscore to remove the effect of outliers from the analysis]
                    z = stats.zscore(max_hitTimes)
                    max_hitTimes = max_hitTimes[(z>-1.6)&(z < 1.2)]
                    max_charge = max_charge[(z>-1.6)&(z < 1.2)]

                    if len(max_hitTimes) < 10:
                        print('exit3')
                        continue

                    #Shifting mean to zero
                    max_hitTimes_mean = sum(max_hitTimes*max_charge)/sum(max_charge)
                    timestamps = max_hitTimes - max_hitTimes_mean
                    final_mean = timestamps.mean()

                    '''
                    Histogramming the data from simulation
                    '''
                    print('Now Histogramming')
                    bins = np.arange(min(timestamps), max(timestamps), 1)
                    num, bin_edges = np.histogram(timestamps, bins=bins, weights=max_charge)
                    bin_centers = (bin_edges[:-1]+bin_edges[1:])/2
                    
                    #removing bins with < 0 entries ---> Recommended for binned likelihood
                    entries_in_bins = num[num > 0]
                    bin_centers = bin_centers[num > 0]

                    #Degrees of freedom should be greater than zero!
                    if len(entries_in_bins) < 8:
                        continue

                    #Getting data for the chi2 fit
                    chi2_entries_in_bins = entries_in_bins[entries_in_bins > 10]
                    chi2_bin_centers = bin_centers[entries_in_bins > 10]

                    num_dataPoints = len(entries_in_bins)
                    area_data = sum(entries_in_bins)
                    mean_entries = entries_in_bins.mean()

                    '''
                    Fitting bifurcated Gaussian and double bifurcated gaussian to
                    the mcpe hit time distributions for both tau and electron.
                    '''

                    #Single Peak

                    nll = lambda *args: log_likelihood_biGauss(*args)
                    initial_biGauss = np.array([final_mean, 50, 5, max(entries_in_bins)])
                    bnds_biGauss = ((min(bin_centers), max(bin_centers)), (0, 500), (0, 10), (0, 1e6))
                    soln_biGauss = minimize(log_likelihood_biGauss, initial_biGauss,
                                            args=(entries_in_bins, bin_centers),
                                            method='Powell',
                                            bounds = bnds_biGauss)

                    #Double Peak

                    nll = lambda *args: log_likelihood_doublePeak(*args)
                    initial_doublePeak = np.array([min(bin_centers)+10, 20, 1, max(entries_in_bins), final_mean, 20, 1, max(entries_in_bins)])
                    bnds_doublePeak = ((min(bin_centers), final_mean), (0, 500), (0, 10), (0, 1e6),
                                        (final_mean, max(bin_centers)), (0, 500), (0, 10), (0,1e6))
                    soln_doublePeak = minimize(log_likelihood_doublePeak, initial_doublePeak,
                                                args=(entries_in_bins, bin_centers),
                                                method='Powell',
                                                bounds=bnds_doublePeak)
                    
                    '''
                    (x, y) values for the fit
                    '''
                    #x = bin_centers
                    x = np.linspace(min(bin_centers), max(bin_centers), 1000)
                    y_biGauss = biGauss(x, soln_biGauss.x[0],
                                            soln_biGauss.x[1], soln_biGauss.x[2], soln_biGauss.x[3])
                    y_doublePeak = double_peak(x, soln_doublePeak.x[0], soln_doublePeak.x[1],
                                                        soln_doublePeak.x[2], soln_doublePeak.x[3], soln_doublePeak.x[4],
                                                        soln_doublePeak.x[5], soln_doublePeak.x[6], soln_doublePeak.x[7])
                    
                    plt.figure(figsize=(10,9))
                    _ = plt.hist(timestamps, bins=bins, weights=max_charge, histtype='step')
                    plt.plot(x, y_biGauss, '-', c = 'k')
                    plt.plot(x, y_doublePeak, '-', c = 'r')
                    plt.title(str(lepton.type)+' '+ str(select_gof_DOM_biGauss) +' '+ str(select_gof_DOM_dp))
                
            frame_number += 1    
    num += 1